In [1]:
import requests
import json
import sys

class RequestHandeler:

    '''For the moment only does GET requests'''

    '''

    INITIALISATION method
        params: string                 : host_url : domain
        params: dict                   : args     : arguments to complete the url
        return: RequestHandeler_Object :          : client connecting to the server
    '''
    def __init__(self, host_url):
        self.host = host_url
        #self.args = args
        #self.url = self.prepare_request()
        #self.url = "http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/ilc_pees01?precision=1&sex=T&yn_rskpov=NO_ARP&yn_rskpov=YES_ARP&unit=PC&workint=WI0-02&workint=WI02-1_NAP&lev_depr=NSEV&lev_depr=SEV&age=TOTAL"
        
        ## NB: not all possible responses for all methods are present in the status variable
        # However, all possible responses for GET requests should be present
        self.status = {
                    100 : ("Succesful response","[+] Connection accepted, cotinue with the request - ignore if the request already returned"),
                    102 : ("Succesful response","[*] Processing the request"),
                    200 : ("Succesful response","[+] OK! Data recieved"),
                    201 : ("Succesful response","[+] POST/PUT Request successful"),
                    202 : ("Succesful response","[*-] Warning: Non-autoritative information - response recieved but not complete"),
                    204 : ("Succesful response","[-] No Contenet: check the request"),
                    206 : ("Succesful response","[*-] Warning: Partial Contenent"),
                    208 : ("Succesful response","[*-] Warning: Milti-Status: multiple status code might be appropriate"),
                    301 : ("Redirection message","[-] Moved Permanently: the requested URL changed - new URL given in te response"),
                    302:  ("Redirection message","[-] Found: requested URL chenged temporaly"),
                    303 : ("Redirection message", "[-] See Other: do a GET request to another URL"),
                    400 : ("Client error","[-] Bad Request: check the request"),
                    401 : ("Client error","[-] Unauthorized: auhentification required"),
                    403 : ("Client error", "[-] Forbidden: no access rights to the contenets"),
                    404 : ("Client error", "[-] Not Found: the server can not fin the reqeusted resource - check the request"),
                    405 : ("Client error", "[-] Method Not Allowed: the request method you used has been deabled by the server - did you try a DELETE ?"),
                    406 : ("Client error","[-] Not Accepted: user-agents do not sayisfy the criteria"),
                    409 : ("Client error","[-] Conflict: request conflicts with the current state of the server"),
                    410 : ("Client error", "[-] Gone: resources no longer"),
                    411 : ("Client error", "[-] Length Required: Contnet-Length field required in the header"),
                    414 : ("Client error", "[-] URL Too Long: URL reuested longer than server can interpret"),
                    415 : ("Client error", "[-] Unsupported Media Type: the media information is not supported by the server"),
                    422 : ("Client error", "[-] Unprocesseble Entity"),
                    423 : ("Client error", "[-] Locked"),
                    428 : ("Client error", "[-] Upgrade Required: upgrade to a different protocol"),
                    429 : ('Client error', "[-] Too Many Requests: too many requests in a given amount of time"),
                    431 : ("Client error", "[-] Request Header Fields Too Large"),
                    500 : ("Server error", "[-] Internal Server Error: the server encontered an error that doen not know how to handle"),
                    501 : ("Server error", "[-] Not Implemented: the request method is not supported by the server"),
                    502 : ("Server error", "[-] Bad Gateway"),
                    503 : ("Server error", "[-] Service Unavailable: service not ready to handle the request"),
                    505 : ("Server error", "[-] HTTP Version Not Supported"),
                    511 : ("Server error", "[-] Network Autherntification Required: the clitns need top authenticate to gain network access")
        }


    '''
    Prepare the request
        params: RequestHandeler_Object
        return: string                 : url : full URL for the GET request
    '''
    def prepare_request(self):
        url = self.host + self.args['table'] + "?"
        for key in self.args:
            if key != 'table':
                if len(self.args[key].split(',')) > 1:
                    for item in self.args[key].split(','):
                        url += "%s=%s&" %(key, item)
                else:
                    url += "%s=%s&" %(key, self.args[key])
        return url[:-1]

    '''
    Do the get request
        params: RequestHandeler_Object
        return:  dict                  : data :    jason dict
        return : string                : datastr : jason string
        return : integer               : response : server response
    '''
    def get_request(self, args):
        self.args = args
        self.url = self.prepare_request()
        r = requests.get(self.url)
        for key in self.status:
            if key == r.status_code:
                message = self.status[key][1]
                print(message)
                print("\t%s" % (self.status[key][0]))
                if '[-]' in message:
                    print('\tData not extracted successfully')
                    print('Exiting ...')
                    sys.exit()
        # json dict
        #data = json.loads(r.content.decode('utf8').replace("'", '"'))
        data = json.loads(r.text)
        #jsons string
        datastr = json.dumps(data, indent=4, sort_keys=True)
        return data, datastr, r.status_code
'''
From separated arguments to dict
    params:  str :  table :    'key = value' form
    params:  str:   optonal args in the same form as previous
    return:  dict:  url_args:  arguments for the URL
    
'''
def args_to_dict(table, *args, **kwargs):
    url_args = {}
    url_args['table'] = table.split("=")[1].strip(' ')
    for ar in args:
        l = ar.split("=")
        url_args[l[0].strip(" ")] = l[1].strip(" ")
    return url_args

def subjason_to_DataFrame(lines, column, subDict):
    M = np.zeros((len(lines), len(column)))
    for i in range(len(lines)):
        for j in range(len(column)):
            M[i][j] = subDict[column[j]][lines[i]]
    return pd.DataFrame(M, index=lines, columns=column)

def json_to_data(lines, columns, values, remove_lines=False, remove_list=[]):
    if not remove_lines and remove_list:
        print('[-] Invalid arguments, Cannot set remove_list when remove_lines is set to False')
        sys.exit()
    d = {}
    j = 0
    for i in range(len(lines)):
        for k in range(len(columns)):
            if lines[i] not in remove_list:
                d[lines[i], columns[k]]= values[j]
            j += 1
    if remove_lines == True:
        for item in remove_list:
            lines.remove(item)
    output = {}
    for i in range(len(columns)):
        output[columns[i]] = {}
        for j in range(len(lines)):
            output[columns[i]][lines[j]] = d[(lines[j], columns[i])]
    return output, lines

### Example
host_url = "http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/"
args = args_to_dict('table = gov_10dd_edpt1', 'na_item = GD')
client = RequestHandeler(host_url)
data , strdata, response = client.get_request(args)
#print(response)
#print(data.keys())

[+] OK! Data recieved
	Succesful response


# Public Balance 2018 and 2019

In [2]:
import matplotlib.pyplot as plt
import numpy as np
#%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]

#gov_10dd_edpt1?na_item=B9&precision=1&unit=PC_GDP&time=2018&time=2019&sector=S13
args = args_to_dict('table= gov_10dd_edpt1', 'na_item=B9', 'precision=1', 'unit=PC_GDP', 'time=2018,2019','sector=S13')
client = RequestHandeler(host_url)
#print(client.url)
data , strdata, response = client.get_request(args) 
times = list(list(data['dimension']['time']['category'].values())[1].values())
#print(times)
country = list(list(data['dimension']['geo']['category'].values())[1].values())
#print(country)
values = list(data['value'].values())
#print(values)
noCountry = ['European Union - 27 countries (2007-2013)' ,
             'European Union - 25 countries (2004-2006)',
             'Euro area - 18 countries (2014)',
             'Euro area - 17 countries (2011-2013)' 
            ]

#d = {}
#j = 0
#for i in range(len(country)):
#    for k in range(len(times)):
#        if country[i] not in noCountry:
#            d[country[i], times[k]]= values[j]
#        j += 1
#for item in noCountry:
#    country.remove(item)


#years = {}
#for i in range(len(times)):
#    years[times[i]] = {}
#    for j in range(len(country)):
#        years[times[i]][country[j]] = d[(country[j], times[i])]       

years, country = json_to_data(country, times, values, remove_lines=True, remove_list=noCountry)
pos = np.arange(len(country))
#print(len(country))
fig, ax = plt.subplots()
ax.set_xticks(pos)
ax.set_xticklabels(country, rotation = 90)
width = 0.35

#print(list(years['2018'].values()))
#print(pos)
ax.bar(pos -  width/2., list(years['2018'].values()),  width, label='2018')
ax.bar(pos +  width/2., list(years['2019'].values()),  width, label='2019')
plt.legend(loc='upper right')
#ax.bar(pos+0.5, list(years['2019'].values()))
plt.show()

[+] OK! Data recieved
	Succesful response


<Figure size 1000x500 with 1 Axes>

In [3]:
import pandas as pd

columns = country
lines = times

M = np.zeros((len(country), len(times)))
for i in range(len(country)):
    for j in range(len(times)):
        M[i][j] = years[times[j]][country[i]]
df = pd.DataFrame(M, index=country, columns=times)
#df.head()



D = subjason_to_DataFrame(country, times, years)
D.head()

,2018,2019
Austria,0.2,0.7
Belgium,-0.8,-1.9
Bulgaria,2.0,2.1
Cyprus,-3.7,1.7
Czechia,0.9,0.3



# Public Balance and General Governement Debt, 2016-2019

In [109]:
args = args_to_dict('table= gov_10dd_edpt1', 
                    'na_item=B9', 
                    'precision=1', 
                    'unit=PC_GDP', 
                    'time=2016,2017,2018,2019',
                    
                    'sector=S13')

data , strdata, response = client.get_request(args) 
times = list(list(data['dimension']['time']['category'].values())[1].values())
#print(times)
country = list(list(data['dimension']['geo']['category'].values())[1].values())
#print(country)
values = list(data['value'].values())
#print(values)
noCountry = ['European Union - 27 countries (2007-2013)' ,
             'European Union - 25 countries (2004-2006)',
             'Euro area - 18 countries (2014)',
             'Euro area - 17 countries (2011-2013)' 
            ]
#print(client.url)


years, country = json_to_data(country, times, values, remove_lines=True, remove_list=noCountry)
D = subjason_to_DataFrame(country, times, years)
print(D)

[+] OK! Data recieved
	Succesful response
                                                  2016  2017  2018  2019
Austria                                           -1.5  -0.8   0.2   0.7
Belgium                                           -2.4  -0.7  -0.8  -1.9
Bulgaria                                           0.1   1.1   2.0   2.1
Cyprus                                             0.3   2.0  -3.7   1.7
Czechia                                            0.7   1.5   0.9   0.3
Germany (until 1990 former territory of the FRG)   1.2   1.2   1.9   1.4
Denmark                                            0.1   1.8   0.7   3.7
Euro area - 19 countries  (from 2015)             -1.5  -1.0  -0.5  -0.6
Estonia                                           -0.5  -0.8  -0.6  -0.3
Greece                                             0.5   0.7   1.0   1.5
Spain                                             -4.3  -3.0  -2.5  -2.8
European Union - 27 countries (from 2020)         -1.3  -0.8  -0.4  -0.6
European 